In [ ]:
import warnings

warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno


from matplotlib.ticker import PercentFormatter
plt.rcParams.update({ "figure.figsize" : (8, 5),"axes.facecolor" : "white", "axes.edgecolor":  "black"})
plt.rcParams["figure.facecolor"]= "w"
pd.plotting.register_matplotlib_converters()
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
import psycopg2
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

#read the database string from the .env
load_dotenv() # aufrufen der .env datei

DB_STRING = os.getenv('DB_STRING') #abfragen der .env informationen 
stri_db = create_engine(DB_STRING) # Aufbau einer verbindung zur DB

# Hier Fetche ich die ge-jointe Tabelle 
query_string = "SET SCHEMA 'eda'; SELECT * FROM eda.king_county_house_details d LEFT JOIN king_county_house_sales s ON d.id = s.house_id;" # SQL befehl zum joinen der Tabellen
db_request = pd.read_sql(query_string, stri_db) # Die Daten Fetchen 
#db_request.to_csv('eda_project_MergedData.csv',index=False) # Speichern der gefetchten und ge-jointen Tabelle

In [ ]:
db_request.head(), db_request.describe(), db_request.columns

In [ ]:
#droping unnecessary or doubled columns
db_request.drop('id', axis=1, inplace=True)
db_request.columns

In [ ]:
# headers are ok

In [ ]:
# check how many duplicated rows exist in the data set - in particular for the 
# house_ids as one house might be sold multiple times

db_request[['house_id', 'price']].duplicated().value_counts()

grouped_df = db_request.groupby(['house_id', 'price'])['date'].count().reset_index(name='anzahl_verkäufe')
doppelte_verkäufe = grouped_df[grouped_df['anzahl_verkäufe'] > 1]
doppelte_verkäufe
# no real duplicates as the duplicated house_ids do not have the same date of sale or price, 
# which means: they just have ben sold multiple times

In [ ]:
# checking for missing data using missingno-libary

msno.bar(db_request)

In [ ]:
# fill na in view
view_imput = int(db_request['view'].mean())

In [ ]:
db_request.fillna({'view': view_imput}, inplace=True)
db_request['view'].isna().sum()

In [ ]:
# dealing with 0 in in renovation year, making them to NaN as easier to plot AND making the years to clear integers
db_request["yr_renovated"] = (db_request["yr_renovated"]) / 10.0
db_request.loc[(db_request.yr_renovated == 0), 'yr_renovated'] = np.nan
db_request['yr_renovated'] = db_request['yr_renovated'].astype(pd.Int64Dtype())
db_request['yr_renovated'].unique()

In [ ]:
# checing for data types
db_request.dtypes

In [ ]:
# change date to timestamp
db_request['date'] = pd.to_datetime(db_request['date'], format='%Y/%m/%d')
type(db_request['date'][0])

In [ ]:
# bedrooms zu int
db_request = db_request.astype({'bedrooms': int})

# view zu int
db_request = db_request.astype({'view': int})

# prices integer
db_request = db_request.astype({'price': int})

In [ ]:
# waterfront 1 zu true und 0 zu false
db_request['waterfront'].fillna(0, inplace=True)
db_request['waterfront'] = db_request['waterfront'].map({1: True, 0: False})
db_request['waterfront'].head()

In [ ]:
# zum speichern der gereinigten daten 
db_request.to_csv('./data/clean_eda_project_MergedData.csv',index=False)